# Yelp Dataset Analysis: Philadelphia Restaurants

Research Question
This project asks: **What factors explain why some Philadelphia restaurants stayed open while others
closed?**

Restaurant survival isn’t just about business success. It also reflects the strength of local
communities. I’m using Yelp data together with neighborhood income information to see how
factors like location, wealth, and customer engagement relate to whether restaurants remain open.
At first, I thought wealthier ZIP codes would show higher survival rates, but the data tells a more
mixed story. Some middle- and lower-income areas actually have more restaurants that stayed open,
possibly because they depend more on local customers or have lower operating costs. This suggests
that community engagement, things like frequent reviews, check-ins, and tips, might matter more
than income alone.

In [1]:
import altair as alt

alt.themes.enable('default')

# alt.themes.set_theme('none')  # makes sure custom config applies

alt.renderers.enable('default')

alt.data_transformers.disable_max_rows()

# Global text settings
alt.default_config = {
    'axis': {
        'labelFontSize': 16,
        'titleFontSize': 18
    },
    'legend': {
        'labelFontSize': 16,
        'titleFontSize': 18
    },
    'title': {
        'fontSize': 22
    }
}


/var/folders/vp/npcvxcc52xqfjgmswcdds6yc0000gn/T/ipykernel_2916/2160748623.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  alt.themes.enable('default')


## Data Loading and Preprocessing

I use three parts of the Yelp Open Dataset: the business file `yelp_academic_dataset_business.json`, the check-in file `yelp_academic_dataset_checkin.json`, and the tip file `yelp_academic_dataset_tip.json`. The business file provides each restaurant’s basic information, ratings, review counts, and open status. The check-in file contains timestamped user check-ins, which I convert into a simple count. The tip file contains short user tips, which I summarize into a tip count and an average compliment count.


I merge these three datasets, keep only restaurants, fill missing engagement values with zeros, and standardize ZIP codes and city names. I then merge in ZIP-level income information from `income-zip.csv` and filter the data to restaurants located in Philadelphia. After removing rows without coordinates, I aggregate to the ZIP level to compute restaurant counts, the share still open, and average engagement metrics. I then merge this summary with the ZIP boundary GeoJSON file `zipcodes_poly.geojson` to prepare the data for mapping and save all cleaned tables for later analysis.

Below is a detailed workflow: 

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("../data")

# Load Yelp business file and keep restaurants
# We keep the following columns from `yelp_academic_dataset_business.json`: 
# `business_id`, `name`, `address`, `city`, `state`, `postal_code`, `latitude`, 
# `longitude`, `stars`, `review_count`, `is_open`, `attributes`, `categories`, `hours`.
biz_path = DATA_DIR / "yelp_academic_dataset_business.json"
biz = pd.read_json(biz_path, lines=True)
biz = biz[biz["categories"].str.contains("Restaurant", na=False)].copy()


# Load Yelp check-in file and summarize
checkin_path = DATA_DIR / "yelp_academic_dataset_checkin.json"
checkin = pd.read_json(checkin_path, lines=True)

# Each row has business_id and a comma-separated list of timestamps in "date"
checkin["checkin_count"] = checkin["date"].apply(
    lambda x: len(x.split(", ")) if isinstance(x, str) else 0
)
checkin = checkin[["business_id", "checkin_count"]]

# Load Yelp tip file and summarize per business
tip_path = DATA_DIR / "yelp_academic_dataset_tip.json"
tip = pd.read_json(tip_path, lines=True)

tip_summary = (
    tip.groupby("business_id")
       .agg(
           tip_count=("text", "count"),
           avg_compliments=("compliment_count", "mean")
       )
       .reset_index()
)


# Merge Yelp business + checkin + tip
yelp = (
    biz.merge(checkin, on="business_id", how="left")
       .merge(tip_summary, on="business_id", how="left")
       .copy()
)

yelp["checkin_count"]   = yelp["checkin_count"].fillna(0).astype(int)
yelp["tip_count"]       = yelp["tip_count"].fillna(0).astype(int)
yelp["avg_compliments"] = yelp["avg_compliments"].fillna(0.0)
yelp["city"] = yelp["city"].astype(str).str.lower()
yelp["postal_code"] = yelp["postal_code"].astype(str).str[:5].str.zfill(5)

# Load Income by ZIP 
inc_path = DATA_DIR / "income-zip.csv"  
income = pd.read_csv(inc_path)


rename_map = {
    "ZIP": "zip",
    "Total population": "population",
    "Households - Median income (dollars)": "median_income",
    "Households - Mean income (dollars)": "mean_income",
}
income = income.rename(columns=rename_map)


income["zip"] = income["zip"].astype(str).str.zfill(5)

# Merge income onto Yelp by ZIP
merged = yelp.merge(income, left_on="postal_code", right_on="zip", how="left")


# Filter to Philadelphia city proper
philly = merged[merged["city"] == "philadelphia"].copy()


philly = philly.dropna(subset=["latitude", "longitude"])
fill_zeros = ["checkin_count", "tip_count", "avg_compliments"]
philly[fill_zeros] = philly[fill_zeros].fillna(0)

if philly["median_income"].isna().any():
    philly["median_income"] = philly["median_income"].fillna(philly["median_income"].median())

# Aggregate to ZIP for choropleths and city summaries
zip_summary = (
    philly.groupby("postal_code")
          .agg(
              n_businesses = ("business_id", "nunique"),
              pct_open     = ("is_open", "mean"),
              avg_stars    = ("stars", "mean"),
              avg_reviews  = ("review_count", "mean"),
              avg_checkins = ("checkin_count", "mean"),
              avg_tips     = ("tip_count", "mean"),
              avg_comps    = ("avg_compliments", "mean"),
              median_income= ("median_income", "mean")
          )
          .reset_index()
)

# Join ZIP polygons for mapping and save reusable files
geo_path = DATA_DIR / "zipcodes_poly.geojson"  # OpenDataPhilly "ZIP Codes – Polygon (GeoJSON)"
geo = gpd.read_file(geo_path)
geo["CODE"] = geo["CODE"].astype(str).str.zfill(5)

merged_geo = geo.merge(zip_summary, left_on="CODE", right_on="postal_code", how="left")

# Save quick artifacts so you can reload without recomputing
DATA_DIR.mkdir(parents=True, exist_ok=True)
philly_out      = DATA_DIR / "philly_yelp_income_clean.csv"
zip_summary_out = DATA_DIR / "philly_zip_summary.csv"
geo_out         = DATA_DIR / "merged_philly.geojson"

philly.to_csv(philly_out, index=False)
zip_summary.to_csv(zip_summary_out, index=False)
merged_geo.to_file(geo_out, driver="GeoJSON")

print("Rows in Yelp restaurants:", len(yelp))
print("Rows in Philadelphia subset:", len(philly))
print("ZIPs in summary:", len(zip_summary))
print("Geo polygons merged:", len(merged_geo))
print("Saved:", philly_out.name, zip_summary_out.name, geo_out.name)

Rows in Yelp restaurants: 52286
Rows in Philadelphia subset: 5856
ZIPs in summary: 57
Geo polygons merged: 48
Saved: philly_yelp_income_clean.csv philly_zip_summary.csv merged_philly.geojson


### Importing the Philadelphia GeoJson files

In [3]:
import geopandas as gpd

geo = gpd.read_file("../data/merged_philly.geojson")
print(geo.columns.tolist())
print(geo.head(3))


['OBJECTID', 'CODE', 'COD', 'Shape__Area', 'Shape__Length', 'postal_code', 'n_businesses', 'pct_open', 'avg_stars', 'avg_reviews', 'avg_checkins', 'avg_tips', 'avg_comps', 'median_income', 'geometry']
   OBJECTID   CODE  COD   Shape__Area  Shape__Length postal_code  \
0         1  19120   20  1.456207e+07   19887.714114       19120   
1         2  19121   21  1.102598e+07   15728.621590       19121   
2         3  19122   22  5.689181e+06    9599.539345       19122   

   n_businesses  pct_open  avg_stars  avg_reviews  avg_checkins  avg_tips  \
0          88.0  0.795455   3.357955    30.170455     50.761364  5.204545   
1          61.0  0.557377   3.204918    35.557377     47.540984  4.786885   
2         103.0  0.669903   3.815534    63.485437     87.009709  7.553398   

   avg_comps  median_income                                           geometry  
0   0.010085        51993.0  POLYGON ((-75.11107 40.04682, -75.10943 40.045...  
1   0.005941        36208.0  POLYGON ((-75.19227 39.994

## **Visualizaiton 1: Mapping Restaurant Survival and Income by ZIP**

Using the merged Philadelphia ZIP code shapes, I create two side-by-side maps to explore how restaurant survival aligns with neighborhood income. **The first map colors each ZIP by the share of restaurants that remain open**, with tooltips showing the ZIP code, number of restaurants, and percent open. **The second map colors the same ZIP boundaries by median household income**, with tooltips listing the ZIP code and its income value. Viewing these maps together provides a quick spatial comparison between economic conditions and restaurant outcomes across the city.

In [4]:
import altair as alt

geo = alt.Data(
    url='../data/merged_philly.geojson',
    format={'type': 'json', 'property': 'features'}  # <-- important
)

pct_open_map = (
    alt.Chart(geo)
      .mark_geoshape(stroke='white', strokeWidth=0.5)
      .encode(
          color=alt.Color('properties.pct_open:Q', title='Share open',
                          scale=alt.Scale(scheme='greens')),
          tooltip=[
              alt.Tooltip('properties.postal_code:N', title='ZIP'),
              alt.Tooltip('properties.n_businesses:Q', title='# restaurants', format=','),
              alt.Tooltip('properties.pct_open:Q', title='Share open', format='.2f')
          ]
      )
      .properties(title='Philadelphia: Share of Restaurants Still Open by ZIP',
                  width=650, height=450)
      .project('mercator')
)


In [5]:
income_map = (
    alt.Chart(geo)
      .mark_geoshape(stroke='white', strokeWidth=0.5)
      .encode(
          color=alt.Color('properties.median_income:Q', title='Median income ($)',
                          scale=alt.Scale(scheme='blues')),
          tooltip=[
              alt.Tooltip('properties.postal_code:N', title='ZIP'),
              alt.Tooltip('properties.median_income:Q', title='Median income', format=',')
          ]
      )
      .properties(title='Philadelphia: Median Household Income by ZIP',
                  width=650, height=450)
      .project('mercator')
)


In [6]:
combined_chart = (
    alt.hconcat(
        pct_open_map,
        income_map
    )
    .resolve_scale(color='independent')
    .configure_title(
        fontSize=22,      
        font='Arial',
        fontWeight='bold'
    )
)

combined_chart.configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
)

alt.HConcatChart(...)

---
## **Visualization 2: Income vs. Restaurant Survival in Philadelphia**

To check whether neighborhood wealth predicts restaurant survival, I plot median household income against the share of restaurants still open. Note that the incomes are scaled by ten-thousand-dollar units because single dollars have negligible impacts.

In [7]:
import statsmodels.api as sm

zip_summary = zip_summary.copy()
zip_summary['income_10k'] = zip_summary['median_income'] / 10000.0

# =========================================================
# Scatterplot with bubble size and regression line
# =========================================================
income_scatter = (
    alt.Chart(zip_summary)
      .mark_circle()
      .encode(
          x=alt.X('income_10k:Q',
                  title='Median Household Income (in $10,000 units)'),
          y=alt.Y('pct_open:Q',
                  title='Share of Restaurants Still Open'),
          size=alt.Size('n_businesses:Q',
                        title='# of Restaurants'),
          tooltip=[
              'postal_code:N',
              alt.Tooltip('median_income:Q', title='Median Income ($)'),
              alt.Tooltip('income_10k:Q', title='Income (10k units)'),
              alt.Tooltip('pct_open:Q', title='Share Open'),
              alt.Tooltip('n_businesses:Q', title='# Restaurants')
          ]
      )
      .properties(
          title='Income vs Restaurant Survival (ZIP level)',
          width=500,
          height=350
      )
)

income_scatter_chart = income_scatter

# =========================================================
# Multiple regression and substantive effect plot
#    pct_open ~ income_10k + n_businesses + avg_checkins
# =========================================================
predictors = ['income_10k', 'n_businesses', 'avg_checkins']
X = sm.add_constant(zip_summary[predictors])
y = zip_summary['pct_open']

mod_income = sm.OLS(y, X).fit()

print(mod_income.summary())

# R-squared and slope
r2 = mod_income.rsquared
slope_income_10k = mod_income.params['income_10k']
intercept = mod_income.params['const']
print(f'Equation: pct_open = {intercept:.3f} + {slope_income_10k:.3f} * income_10k')
print(f'R^2 = {r2:.3f}')


income_grid = np.linspace(
    zip_summary['income_10k'].min(),
    zip_summary['income_10k'].max(),
    100
)


X_pred_const = pd.DataFrame({
    'const': 1.0,
    'income_10k': income_grid,
    'n_businesses': zip_summary['n_businesses'].mean(),
    'avg_checkins': zip_summary['avg_checkins'].mean()
})


pred = mod_income.get_prediction(X_pred_const)
pred_df = pred.summary_frame(alpha=0.05)  # 95 percent CI

effect_df = pd.DataFrame({
    'income_10k': income_grid,
    'pct_open_pred': pred_df['mean'],
    'lower': pred_df['mean_ci_lower'],
    'upper': pred_df['mean_ci_upper']
})


# Effect plot: line + shaded confidence band
effect_band = (
    alt.Chart(effect_df)
      .mark_area(opacity=0.2)
      .encode(
          x=alt.X('income_10k:Q',
                  title='Median Household Income (in $10,000 units)'),
          y='lower:Q',
          y2='upper:Q'
      )
)

effect_line = (
    alt.Chart(effect_df)
      .mark_line()
      .encode(
          x='income_10k:Q',
          y=alt.Y('pct_open_pred:Q',
                  title='Predicted Share of Restaurants Open')
      )
)

effect_plot = (
    (effect_band + effect_line)
      .properties(
        title=alt.TitleParams(
            text=[
                "Predicted Share of Restaurants Open vs Income",
                "(controls for # restaurants and check-ins)"
            ]
        ),
        width=500,
        height=350
    )
)


combined_chart = (alt.hconcat(income_scatter_chart, effect_plot)     
                  .configure_title(
        fontSize=22,     
        font='Arial',
        fontWeight='bold'
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
))


combined_chart


                            OLS Regression Results                            
Dep. Variable:               pct_open   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     3.820
Date:                Tue, 09 Dec 2025   Prob (F-statistic):             0.0150
Time:                        00:44:36   Log-Likelihood:                 10.387
No. Observations:                  57   AIC:                            -12.77
Df Residuals:                      53   BIC:                            -4.602
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.8397      0.080     10.451   

alt.HConcatChart(...)

### Scatterplot (raw relationship)
The scatterplot shows the raw relationship between neighborhood income and restaurant survival across Philadelphia ZIP codes. Even without a fitted line, the points form a general downward pattern: lower-income ZIP codes tend to have higher survival rates, and higher-income ZIP codes often show lower rates. At the same time, the scatter is wide. ZIP codes with similar income levels can have very different outcomes. For example, around eighty thousand dollars in income, survival ranges from roughly ten percent to about seventy percent. At the low-income end, survival spans from about sixty percent to ninety percent. These large ranges show that income may explain part of the variation in restaurant survival but not much. **Neighborhoods with comparable income levels often end up with different survival rates, which limits how strongly income alone can predict whether restaurants remained open.**

### Substantive effect plot (model-based prediction)
The substantive effect plot shows the model’s predicted share of restaurants open at different income levels, while holding the number of restaurants and average check-ins at their average values. Doing this isolates the role of income itself, since the other neighborhood characteristics are not allowed to vary. The prediction line slopes downward, which reflects the negative income coefficient in the regression. However, the shaded confidence band around the line is wide, especially at the lowest and highest incomes. This widening occurs because there are fewer ZIP codes in those ranges, and it signals substantial uncertainty about the exact size of the decline. **The overall trend still points downward, but the width of the confidence band and the model’s low R² show that income is not a strong or reliable predictor once other neighborhood factors and randomness are taken into account.**


---

## **Visualization 3: Check-ins & Review Counts vs Restaurant Survival**

After examining income and seeing a clear downward pattern in the scatterplot, the substantive effect plot showed that income itself is not a strong or reliable predictor once other factors are held constant. 

To understand whether customer activity might play a larger role, I next look at two additional ZIP-level measures of engagement: **average check-ins** and **average review counts**. Both variables capture different aspects of how often customers interact with restaurants. If more activity helped restaurants survive, we would expect positive relationships. Instead, **both plots reveal similar downward trends.**



### **Check-ins (Customer Activity) vs Restaurant Survival**

Check-ins provide a proxy for foot traffic or on-site customer visits. If higher foot traffic helped restaurants survive, we would expect survival to rise as check-ins increase. Instead, the scatterplot shows a slight downward pattern. The fitted regression line confirms this:

```
pct_open = -0.0009 * check-ins + 0.7673
```

The correlation is modest and negative (r = –0.388). ZIP codes with higher check-in activity do not necessarily show higher survival. In several cases, ZIPs with high check-ins actually fall on the lower end of the survival range. This suggests that customer foot traffic at the ZIP level does not reliably predict whether restaurants remained open.


### **Review Counts (Customer Engagement) vs Restaurant Survival**

Review counts measure how many people interact with restaurants on Yelp. If engagement helped restaurants stay open, we would expect an upward trend. Instead, the scatterplot shows the same downward direction as the check-in plot. The regression slope is again negative:

```
pct_open = -0.0018 * reviews + 0.7878
```

The correlation is nearly identical in size (r = –0.393). ZIPs where restaurants accumulate more reviews do not show higher survival rates. Even areas with strong online engagement can experience below-average survival. Together, these patterns indicate that higher customer activity—whether measured by check-ins or reviews—does not translate into higher restaurant survival.

In [8]:
import altair as alt
import numpy as np

# ---------- Check-ins ----------
checkins_scatter = (
    alt.Chart(zip_summary)
      .mark_circle()
      .encode(
          x=alt.X('avg_checkins:Q', title='Average Check-ins per Restaurant'),
          y=alt.Y('pct_open:Q', title='Share of Restaurants Still Open'),
          size=alt.Size('n_businesses:Q', title='# of Restaurants'),
          tooltip=['postal_code:N','avg_checkins:Q','pct_open:Q','n_businesses:Q']
      )
)

checkins_line = (
    alt.Chart(zip_summary)
      .transform_regression('avg_checkins', 'pct_open')
      .mark_line(color='red', size=2)
      .encode(
          x='avg_checkins:Q',
          y='pct_open:Q'
      )
)

checkins_chart = (checkins_scatter + checkins_line).properties(
    title='Check-ins vs Restaurant Survival (ZIP level)',
    width=400, height=300
)

# ---------- Reviews ----------
reviews_scatter = (
    alt.Chart(zip_summary)
      .mark_circle()
      .encode(
          x=alt.X('avg_reviews:Q', title='Average Review Count'),
          y=alt.Y('pct_open:Q', title='Share of Restaurants Still Open'),
          size=alt.Size('n_businesses:Q', title='# of Restaurants'),
          tooltip=['postal_code:N','avg_reviews:Q','pct_open:Q','n_businesses:Q']
      )
)

reviews_line = (
    alt.Chart(zip_summary)
      .transform_regression('avg_reviews', 'pct_open')
      .mark_line(color='red', size=2)
      .encode(
          x='avg_reviews:Q',
          y='pct_open:Q'
      )
)

reviews_chart = (reviews_scatter + reviews_line).properties(
    title='Review Counts vs Restaurant Survival (ZIP level)',
    width=400, height=300
)

combined_chart = (alt.hconcat(checkins_chart, reviews_chart)     
                  .configure_title(
        fontSize=22,     
        font='Arial',
        fontWeight='bold'
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
))
combined_chart


alt.HConcatChart(...)

In [9]:
def summarize_linear(x, y, label):
    slope, intercept = np.polyfit(x, y, 1)
    corr = x.corr(y)
    print(f"{label}:")
    print(f"  pct_open = {slope:.4f} * x + {intercept:.4f}")
    print(f"  correlation r = {corr:.3f}")
    print()

summarize_linear(zip_summary["avg_checkins"], zip_summary["pct_open"],
                 "Average check-ins")
summarize_linear(zip_summary["avg_reviews"], zip_summary["pct_open"],
                 "Average reviews")


Average check-ins:
  pct_open = -0.0009 * x + 0.7673
  correlation r = -0.388

Average reviews:
  pct_open = -0.0018 * x + 0.7878
  correlation r = -0.393



### Summary of ZIP-level Relationships

Across income, check-ins, and review counts, all three ZIP-level regressions show weak negative trends and modest negative correlations. None of these variables strongly predicts which neighborhoods kept more restaurants open. This suggests that ZIP-level characteristics may not capture the main drivers of restaurant survival, and that restaurant-level factors or category-level differences may explain more of the variation.

---

In [10]:
import geopandas as gpd

# load merged ZIP GeoJSON
gdf = gpd.read_file("../data/merged_philly.geojson")

# project to a metric CRS so areas are meaningful
gdf_proj = gdf.to_crs(epsg=3857)   # Web Mercator; good enough for km²

# area in square kilometers
gdf_proj["area_km2"] = gdf_proj.geometry.area / 1e6

# restaurant density
gdf_proj["restaurants_per_km2"] = gdf_proj["n_businesses"] / gdf_proj["area_km2"]

# keep a plain DataFrame version for Altair scatter
zip_density = gdf_proj[["postal_code", "n_businesses", "pct_open",
                        "area_km2", "restaurants_per_km2"]].copy()


## **Visualizations 4 & 5: Restaurant Density as a Possible Factor**

Restaurant density might matter for survival. High density could increase competition and push weaker restaurants out. High density could also mark a destination dining area where restaurants benefit from shared foot traffic. Because these forces point in opposite directions, I first map how many restaurants each ZIP contains, then plot restaurant count and restaurant density against survival. I displayed the density map first. 

In [11]:
geo_density = alt.Data(
    url="../data/merged_philly.geojson",
    format={"type": "json", "property": "features"}
)

density_map = (
    alt.Chart(geo_density)
      .mark_geoshape(stroke="white", strokeWidth=0.5)
      .encode(
          color=alt.Color("properties.n_businesses:Q",
                          title="# of restaurants",
                          scale=alt.Scale(scheme="blues")),
          tooltip=[
              alt.Tooltip("properties.postal_code:N", title="ZIP"),
              alt.Tooltip("properties.n_businesses:Q",
                          title="# restaurants", format=",")
          ]
      )
      .properties(
          title="Number of restaurants by ZIP in Philadelphia",
          width=650,
          height=450
      )
      .project("mercator")
)

density_map


alt.Chart(...)

The density map shows that restaurants cluster heavily in a few central ZIP codes, while many outer neighborhoods have far fewer places to eat. 

The two scatterplots below add more detail. I plot both the number of restaurants and the number of restaurants per square kilometer against the share that remain open, and I add a LOESS smoother to show the shape of the relationship without forcing it to be linear.

In [12]:
# Restaurant count vs survival
scatter_n = (
    alt.Chart(zip_density)
      .mark_circle()
      .encode(
          x=alt.X("n_businesses:Q", title="# of restaurants in ZIP"),
          y=alt.Y("pct_open:Q", title="Share of restaurants still open"),
          tooltip=["postal_code:N", "n_businesses:Q", "pct_open:Q"]
      )
      .properties(
          width=400,
          height=300
      )
)

loess_counts = (
    alt.Chart(zip_density)
      .transform_loess('n_businesses', 'pct_open', bandwidth=0.5)
      .mark_line(color='red', size=2)
      .encode(x='n_businesses:Q', y='pct_open:Q')
)

(scatter_n + loess_counts)


chart_counts = (
    (scatter_n + loess_counts)
    .properties(title="Restaurant count vs survival (ZIP level)")
)


# Restaurant density vs survival
scatter_density = (
    alt.Chart(zip_density)
      .mark_circle()
      .encode(
          x=alt.X("restaurants_per_km2:Q",
                  title="Restaurants per km²"),
          y=alt.Y("pct_open:Q", title="Share of restaurants still open"),
          tooltip=["postal_code:N", "restaurants_per_km2:Q", "pct_open:Q"]
      )
      .properties(
          width=400,
          height=300
      )
)

loess_density = (
    alt.Chart(zip_density)
      .transform_loess('restaurants_per_km2', 'pct_open', bandwidth=0.5)
      .mark_line(color='red', size=2)
      .encode(x='restaurants_per_km2:Q', y='pct_open:Q')
)

(scatter_density + loess_density)


chart_density = (
    (scatter_density + loess_density)
    .properties(title="Restaurant density vs survival (ZIP level)")
)



combined_chart = (alt.hconcat(chart_counts, chart_density)     
                  .configure_title(
                        fontSize=22,     
                        font='Arial',
                        fontWeight='bold'
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
))
combined_chart

alt.HConcatChart(...)

**What the Density Plots Show**

The patterns are similar for both measures. **ZIP codes with fewer restaurants or low density tend to have higher survival rates.** Survival is often around seventy to eighty percent in these areas. As counts and density rise into the midrange, the LOESS curves slope downward and survival drops into the mid fifties. Beyond that point, the lines flatten. Among the densest ZIP codes, survival values cluster in a narrow midrange rather than continuing to fall. This suggests that density has a limited effect. **Density helps explain why some low-density neighborhoods have strong survival, but it does not do much to distinguish outcomes once ZIP codes become moderately or highly dense.**

---

## **Visualization 6: Restaurant Survival by Category and Chain Status**

Restaurant type is one of the clearest ways to explain differences in survival. Different categories face different costs, customer expectations, and levels of competition. Pizza, Mexican, and Chinese places often have steady demand and relatively simple menus. Full-service American restaurants may rely more on discretionary spending and have higher labor and overhead. 

Chain affiliation can also shape resilience. Chains can draw on corporate resources, brand recognition, and standardized operations. Independent restaurants tend to lean more on neighborhood foot traffic and local loyalty.

In [13]:
# Category: Simplify categories into broader groups

def simplify_category(cat_string):
    """
    Simplify a Yelp categories string into a broader category.
    """
    if pd.isna(cat_string):
        return "Other"
    
    cat_string = cat_string.lower()
    
    if "pizza" in cat_string:
        return "Pizza"
    if "chinese" in cat_string:
        return "Chinese"
    if "mexican" in cat_string:
        return "Mexican"
    if "coffee" in cat_string or "cafe" in cat_string:
        return "Coffee & Tea"
    if "bar" in cat_string or "pub" in cat_string:
        return "Bars & Pubs"
    if "fast food" in cat_string or "burgers" in cat_string:
        return "Fast Food"
    if "american" in cat_string:
        return "American"
    
    return "Other"

philly["category_simple"] = philly["categories"].apply(simplify_category)

# Category summary
cat_summary = (
    philly.groupby("category_simple")
          .agg(
              pct_open=("is_open", "mean"),
              n=("business_id", "count")
          )
          .reset_index()
)

In [14]:
# Chain: To simplify the definition, I identify chains by duplicated names. 
philly["is_chain"] = philly["name"].duplicated(keep=False)
chain_summary = (
    philly.groupby("is_chain")
          .agg(
              pct_open=("is_open", "mean"),
              n=("business_id", "count")
          )
          .reset_index()
)

chain_summary["chain_label"] = chain_summary["is_chain"].map({True: "Chain", False: "Independent"})


Before plotting, I simplify Yelp’s detailed category strings into a small set of broader groups such as Pizza, Mexican, Chinese, Coffee & Tea, Bars & Pubs, Fast Food, American, and Other. I then compute, for each category, how many restaurants fall into that group and what share remain open. 

A similar summary separates restaurants into chains and independents by flagging names that appear in multiple ZIP codes. These summary tables ensure that the plotted differences reflect hundreds of restaurants per group rather than a few outliers.


In [15]:
# Category summary table (sorted by sample size)
category_summary_display = (
    cat_summary[["category_simple", "n", "pct_open"]]
    .sort_values("n", ascending=False)
)

category_summary_display


,category_simple,n,pct_open
6,Other,1679,0.574151
1,Bars & Pubs,1124,0.588968
7,Pizza,800,0.661250
3,Coffee & Tea,701,0.629101
2,Chinese,467,0.635974
0,American,450,0.506667
4,Fast Food,335,0.626866
5,Mexican,300,0.656667


In [16]:
# Chain vs independent summary table
chain_summary_display = chain_summary.copy()
chain_summary_display

,is_chain,pct_open,n,chain_label
0,False,0.586482,4764,Independent
1,True,0.672161,1092,Chain


Plots are as follow:

In [17]:
cat_chart = (
    alt.Chart(cat_summary)
      .mark_bar()
      .encode(
          x=alt.X("pct_open:Q", title="Share of Restaurants Still Open"),
          y=alt.Y("category_simple:N", sort="-x", title="Category"),
          color=alt.Color("pct_open:Q", scale=alt.Scale(scheme="greens")),
          tooltip=["category_simple:N", "pct_open:Q", "n:Q"]
      )
      .properties(
          title="Restaurant Survival by Category (Philadelphia)",
          width=450,
          height=300
      )
)


In [18]:
chain_chart = (
    alt.Chart(chain_summary)
      .mark_bar()
      .encode(
          x=alt.X("pct_open:Q", title="Share Open"),
          y=alt.Y("chain_label:N", sort="-x", title="Restaurant Type"),
          color=alt.Color("pct_open:Q", scale=alt.Scale(scheme="blues")),
          tooltip=["chain_label:N", "pct_open:Q", "n:Q"]
      )
      .properties(
          title="Survival Rates: Chains vs Independents",
          width=350,
          height=200
      )
)

chain_chart



combined_chart = alt.hconcat(
    cat_chart,
    chain_chart
).resolve_scale(
    color='independent'
).configure_title(
    fontSize=22,     
    font='Arial',
    fontWeight='bold'
)

combined_chart.configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
)


alt.HConcatChart(...)

**Interpretation:**

The left panel shows survival rates by category. Pizza and Mexican restaurants have the highest survival, with about two-thirds still open. Chinese, Coffee & Tea, Fast Food, and Bars & Pubs cluster just below that level. American restaurants stand out as the lowest category, with only about half still open. 

The right panel compares chains and independents. Chains have a survival rate near 67 percent, while independents are closer to 59 percent. Combined, these patterns suggest that both category and chain status matter. Some types of restaurants, especially pizza and Mexican places and chain locations, appear structurally more likely to survive.

---

## **Combined Analysis: Bringing All Predictors Together** 

*Note: this is an optional section that I am exploring.*

Up to this point, each factor—income, density, check-ins, reviews, and restaurant type—was examined separately. These single-variable plots help show raw patterns, but they cannot tell whether a factor still matters after accounting for the others. To understand which predictors contribute unique explanatory power, I fit a multiple regression that includes all major variables at the ZIP level:
- Median income (in $10k units)

- Restaurant density (restaurants per km²)

- Average check-ins

- Average reviews

- A chain-share measure

- Broad restaurant-category indicators

This model asks a more complete question:

**When ZIP codes differ on several dimensions at once, which factors still help explain variation in survival rates?**

In [19]:
# Rerun previous cells to ensure philly, zip_summary, and zip_density are defined
philly = philly.copy()
philly['postal_code'] = philly['postal_code'].astype(str).str.zfill(5)

zip_summary = zip_summary.copy()
zip_summary['postal_code'] = zip_summary['postal_code'].astype(str).str.zfill(5)

zip_density = zip_density.copy()
zip_density['postal_code'] = zip_density['postal_code'].astype(str).str.zfill(5)

zip_cat_chain = (
    philly.groupby('postal_code')
          .agg(
              chain_share=('is_chain', 'mean'),  # fraction of restaurants that are chains
              main_category=('category_simple',
                             lambda x: x.value_counts().idxmax())  # most common category
          )
          .reset_index()
)


In [20]:
# Rerun merges to create modeling DataFrame
model_df = (
    zip_summary
      .merge(zip_density[['postal_code', 'restaurants_per_km2']],
             on='postal_code', how='left')
      .merge(zip_cat_chain,
             on='postal_code', how='left')
)

if 'income_10k' not in model_df.columns:
    model_df['income_10k'] = model_df['median_income'] / 10000.0

# Drop rows with missing values in key predictors
model_df = model_df.dropna(subset=[
    'pct_open',
    'income_10k',
    'restaurants_per_km2',
    'avg_checkins',
    'avg_reviews',
    'chain_share',
    'main_category'
])


In [21]:
# One-hot encode the dominant category per ZIP (drop one as reference)
cat_dummies = pd.get_dummies(model_df['main_category'],
                             prefix='cat', drop_first=True)


X = pd.concat([
    model_df[['income_10k',
              'restaurants_per_km2',
              'avg_checkins',
              'avg_reviews',
              'chain_share']],
    cat_dummies
], axis=1)


df_reg = X.join(model_df['pct_open']).dropna()

y = df_reg['pct_open']
X = df_reg.drop(columns=['pct_open'])

X = sm.add_constant(X)

col_names = X.columns.tolist()

In [22]:
X_np = X.to_numpy(dtype=float)
y_np = y.to_numpy(dtype=float)

mod_full = sm.OLS(y_np, X_np).fit()
print(mod_full.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     9.274
Date:                Tue, 09 Dec 2025   Prob (F-statistic):           3.43e-07
Time:                        00:44:36   Log-Likelihood:                 58.220
No. Observations:                  47   AIC:                            -96.44
Df Residuals:                      37   BIC:                            -77.94
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9621      0.090     10.661      0.0

### Findings from the Multiple Regression

Most predictors, including income, density, and check-ins, remain statistically insignificant once the other variables are included. Their coefficients are small, which means they only shift predicted survival rates slightly when they change.

The one variable that reaches statistical significance is:

**• Average Reviews (p ≈ 0.018)**


ZIP codes with more highly reviewed restaurants tend to have slightly lower survival rates, though the effect size is still modest. This matches the earlier scatterplots, where more active or popular areas often showed lower survival because these ZIPs include competitive, high-traffic districts where turnover is common.


Overall, the combined model reinforces that no single neighborhood factor has a strong or consistent effect once everything is controlled simultaneously.

In [23]:
coef_table = pd.DataFrame({
    "coef": pd.Series(mod_full.params, index=col_names),
    "p_value": pd.Series(mod_full.pvalues, index=col_names)
}).sort_values("coef")

coef_table_rounded = coef_table.round({
    "coef": 3,
    "p_value": 3
})
coef_table_rounded

,coef,p_value
cat_Fast Food,-0.126,0.216
cat_Coffee & Tea,-0.094,0.312
chain_share,-0.058,0.760
cat_Pizza,-0.050,0.388
cat_Other,-0.047,0.232
income_10k,-0.009,0.348
avg_reviews,-0.004,0.018
restaurants_per_km2,0.001,0.111
avg_checkins,0.001,0.282
const,0.962,0.000


In [24]:
coef_df = (
    coef_table_rounded.drop(index='const')      # drop intercept
              .assign(abs_coef=lambda d: d['coef'].abs())
              .sort_values('abs_coef', ascending=True)
              .reset_index()
              .rename(columns={'index': 'variable'})
)

importance_plot = (
    alt.Chart(coef_df)
      .mark_bar()
      .encode(
          x='abs_coef:Q',
          y=alt.Y('variable:N', sort='-x'),
          tooltip=['variable', 'coef', 'p_value']
      )
      .properties(
          title='Relative Importance of Predictors (Combined Regression)',
          width=400,
          height=300
      )
)

importance_plot.configure_title(
    fontSize=22,     
    font='Arial',
    fontWeight='bold'
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=18
)


alt.Chart(...)

**Feature-Importance Results:**

The feature-importance chart compares predictors by the size of their coefficients. Larger bars mean the predictor moves the model’s predictions more.

The main takeaway is that restaurant category dominates. Category indicators such as Fast Food, Coffee & Tea, and Pizza have the largest coefficients, meaning type of restaurant creates the biggest structural differences in survival. However, none of these category coefficients are statistically significant, so they should be interpreted as patterns in the sample rather than reliable estimates of true effects.

By comparison, income, density, reviews, and check-ins have much smaller coefficients, and they are also not statistically significant once everything is included together. The combined model explains a moderate share of variation, but most of that comes from broad differences across groups rather than any single strong predictor.

# **Conclusion**

**Restaurant survival in Philadelphia does not appear to hinge on any single neighborhood characteristic.** Several factors show patterns, but each explains only part of the variation.

* **Income shows a downward trend, but the regression and substantive-effect plot indicate that the relationship is modest** once other factors are held constant. Higher-income ZIP codes tend to have slightly lower survival, but income alone does not reliably explain which neighborhoods fare better.

* **Customer activity (check-ins and reviews) also shows weak negative correlations.** ZIPs with more Yelp engagement do not have higher survival; in several cases, highly active areas actually fall on the lower end. This implies that visibility and foot traffic do not necessarily translate into resilience.

* **Density matters only at the low end.** ZIP codes with very few restaurants or low density tend to have higher survival rates. Once neighborhoods reach moderate or high density, differences flatten out and survival levels converge.

* **Restaurant type adds a clear structural layer.** Categories such as pizza, Mexican, and coffee shops show higher survival, while American restaurants perform worse. Chains also outperform independents, likely due to brand strength, standardized operations, and better access to resources.

* **When all variables are combined in a single regression, only one predictor—average reviews—remains statistically significant, and its effect is modest.** All other predictors, including income, density, check-ins, chain share, and the category indicators, are not statistically significant once they are included together. The combined model explains some variation, but most coefficients are small and uncertain. This reinforces that survival differences arise from many interacting forces rather than from any single strong driver.

Taken together, these findings show that restaurant survival reflects **a mix of economic, operational, and neighborhood factors**, none of which is sufficient on its own. Income, engagement, density, and category each contribute to the overall picture, but the data also highlight the limits of simple explanations. Real-world outcomes likely depend on additional elements, such as rent costs, competition from nearby commercial corridors, owner experience, and local customer preferences, that fall outside the scope of this dataset. The analysis helps clarify where patterns exist, but it also shows the complexity of what shapes restaurant resilience in a city like Philadelphia.
